# Bank Dataset Project by David Smolinski
What this adds to my portfolio:
- common sql
- csv to pandas
- pandas to sql
- sql to pandas

Links:
- [my portfolio](https://github.com/DavidSmolinski/portfolio)
- [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing)
- [the csv](https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip)

In [12]:
import sqlite3, numpy as np, requests, pandas as pd
from io import BytesIO
from zipfile import ZipFile

# get the csv in the zipped folder at the url
zip_folder_url="https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip"
response = requests.get(zip_folder_url)
with ZipFile(BytesIO(response.content)) as z:
    with z.open('bank.csv') as csv:
        df = pd.read_csv(csv, delimiter=';')

print(df.head(1))

   age         job  marital education default  balance housing loan   contact  \
0   30  unemployed  married   primary      no     1787      no   no  cellular   

   day month  duration  campaign  pdays  previous poutcome   y  
0   19   oct        79         1     -1         0  unknown  no  


The next cell makes the database in RAM, and connects to it. To make or connect to a db file, use: 
connection = sqlite3.connect('path.db')

In [13]:
connection = sqlite3.connect(":memory:")

Method 1 of putting the dataframe (df) into the database (db):

In [14]:
df.to_sql('clients', connection)

Method 2 of putting the dataframe into the database:

In [15]:
c = connection.cursor()

c.execute("""CREATE TABLE IF NOT EXISTS clients (
             "index" SMALLINT PRIMARY KEY,
             age TINYINT,
             job VARCHAR(15),
             marital VARCHAR(10),
             education VARCHAR(10),
             "default" VARCHAR(3),
             balance BIGINT,
             housing VARCHAR(3),
             loan VARCHAR(3),
             contact VARCHAR(10),
             day TINYINT,
             month VARCHAR(3),
             duration SMALLINT,
             campaign TINYINT,
             pdays SMALLINT,
             previous TINYINT,
             poutcome VARCHAR(10),
             y VARCHAR(3)
             )""")


for row in range(len(df)):
    row_tuple = (row,)
    for col in range(len(df.columns)):
        val = df.iloc[row, col]
        if isinstance(val, np.int64):
            val = int(val)
        row_tuple += (val,)
    with connection:
        c.execute(
            """INSERT INTO clients ("index", age, job, marital, education, "default", balance, housing, loan, contact, day, 
            month, duration, campaign, pdays, previous, poutcome, y) VALUES 
            (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            row_tuple)

querying the database:

In [16]:
query_number = 1


def print_column_names(connection, table):
    c = connection.cursor()
    c.execute("""PRAGMA table_info(""" + table + ")")
    table_info = c.fetchall()
    column_names = [e[1] for e in table_info]
    print(f"\n\ncolumn names\n{column_names}")


def query(connection, query_number, sql_code):
    c = connection.cursor()
    c.execute(sql_code)
    print(f"\n\nquery {query_number}:\n\nsql:\n{sql_code}\n\nresult:\n{c.fetchall()}")
    query_number += 1
    return query_number


print_column_names(connection, "clients")

sql_code = """SELECT * FROM clients WHERE "index" < 40 ORDER BY age, balance ASC LIMIT 5"""
query_number = query(connection=connection, query_number=query_number, sql_code=sql_code)

sql_code = """SELECT education AS ed, balance FROM clients 
              WHERE job IN('housemaid', 'entrepreneur', 'student') AND balance between 500 AND 2000"""
query_number = query(connection, query_number, sql_code)

sql_code = """SELECT job, SUM(balance) FROM clients GROUP BY job"""
query_number = query(connection, query_number, sql_code)

sql_code = """SELECT DISTINCT education FROM clients WHERE education LIKE '%r%a_y'"""
query_number = query(connection, query_number, sql_code)

sql_code = """SELECT DISTINCT age FROM clients WHERE education =
              (SELECT DISTINCT education FROM clients WHERE balance >30000)"""
query_number = query(connection, query_number, sql_code)

sql_code = """SELECT DISTINCT age FROM clients WHERE education =
              (SELECT DISTINCT education FROM clients WHERE balance >30000)"""
query_number = query(connection, query_number, sql_code)



column names
['index', 'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']


query 1:

sql:
SELECT * FROM clients WHERE "index" < 40 ORDER BY age, balance ASC LIMIT 5

result:
[(13, 20, 'student', 'single', 'secondary', 'no', 502, 'no', 'no', 'cellular', 30, 'apr', 261, 1, -1, 0, 'unknown', 'yes'), (13, 20, 'student', 'single', 'secondary', 'no', 502, 'no', 'no', 'cellular', 30, 'apr', 261, 1, -1, 0, 'unknown', 'yes'), (39, 23, 'services', 'single', 'tertiary', 'no', 363, 'yes', 'no', 'unknown', 30, 'may', 16, 18, -1, 0, 'unknown', 'no'), (39, 23, 'services', 'single', 'tertiary', 'no', 363, 'yes', 'no', 'unknown', 30, 'may', 16, 18, -1, 0, 'unknown', 'no'), (18, 25, 'blue-collar', 'single', 'primary', 'no', -221, 'yes', 'no', 'unknown', 23, 'may', 250, 1, -1, 0, 'unknown', 'no')]


query 2:

sql:
SELECT education AS ed, balance FROM clients 
              WHERE job IN(

making a dataframe from the database:

In [17]:
sql = "SELECT * FROM clients"
df = pd.read_sql(sql, connection)
print(df.head(3))

   index  age         job  marital  education default  balance housing loan  \
0      0   30  unemployed  married    primary      no     1787      no   no   
1      1   33    services  married  secondary      no     4789     yes  yes   
2      2   35  management   single   tertiary      no     1350     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome   y  
0  cellular   19   oct        79         1     -1         0  unknown  no  
1  cellular   11   may       220         1    339         4  failure  no  
2  cellular   16   apr       185         1    330         1  failure  no  


In [18]:
connection.close()

dataset citations:

[Moro et al., 2011] S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. 
  In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM'2011, pp. 117-121, Guimarães, Portugal, October, 2011. EUROSIS.

  Available at: [pdf] http://hdl.handle.net/1822/14838
                [bib] http://www3.dsi.uminho.pt/pcortez/bib/2011-esm-1.txt